# mcap_dumper コマンド生成ノートブック（クォート自動/なし対応）

このノートブックを実行すると、以下の 3 つのテキストファイルから値を読み込み、`mcap_dumper` の CLI コマンドを生成して表示します。

- `data_dirname.txt` : rosbag データのディレクトリ名（パス）を **1行**
- `topic_name.txt` : topic 名を **1行に1つ**（複数行OK）
- `output_dirname.txt` : 出力ディレクトリ名（パス）を **1行**

## クォート（"...")について

あなたの希望は **クォート無し** ですが、パスにスペースが入る環境だとクォート無しは壊れます。
そのため、このノートブックは **デフォルトで `QUOTE_MODE='auto'`**（スペース等があるときだけクォート）にしています。

- `QUOTE_MODE = 'auto'` : 必要なときだけクォート（おすすめ）
- `QUOTE_MODE = 'never'` : 常にクォートしない（あなたの希望どおり）
- `QUOTE_MODE = 'always'` : 常にクォートする


In [ ]:
from pathlib import Path
import re

# ===== 設定（必要ならここだけ変更） =====
DATA_DIR_FILE = Path('data_dirname.txt')
TOPIC_FILE = Path('topic_name.txt')
OUTPUT_DIR_FILE = Path('output_dirname.txt')

# mcap_dumper のオプション名が環境で違う場合はここを変更
OUTPUT_DIR_FLAG = '--output_dir'   # 例: '--output-dir' など

# クォートの付け方
QUOTE_MODE = 'auto'  # 'auto' / 'never' / 'always'

# ===== ユーティリティ =====
def read_first_nonempty_line(path: Path) -> str:
    """1行目（空白行はスキップ）の文字列を返す。BOMがあってもOK。"""
    if not path.exists():
        raise FileNotFoundError(f"{path} が見つかりません")
    text = path.read_text(encoding='utf-8-sig')  # utf-8-sig でBOM自動除去
    for line in text.splitlines():
        s = line.strip()
        if s:
            return s
    raise ValueError(f"{path} に有効な内容がありません（空行のみ）")

def read_nonempty_lines(path: Path) -> list[str]:
    """空白行を除去した行リストを返す。BOMがあってもOK。"""
    if not path.exists():
        raise FileNotFoundError(f"{path} が見つかりません")
    text = path.read_text(encoding='utf-8-sig')
    return [line.strip() for line in text.splitlines() if line.strip()]

def ensure_leading_slash(topic: str) -> str:
    topic = topic.strip()
    if not topic.startswith('/'):
        topic = '/' + topic
    return topic

def dq(s: str) -> str:
    """ダブルクォートで囲む（中の " は \" にエスケープ）"""
    return '"' + s.replace('"', '\\"') + '"'

def q(s: str) -> str:
    """QUOTE_MODE に応じてクォートする/しない。
    auto の場合は、空白やシェルで問題になりやすい文字が含まれるときだけクォート。
    """
    mode = QUOTE_MODE.lower()
    if mode == 'never':
        return s
    if mode == 'always':
        return dq(s)

    # auto: 空白 or 一部のシェル特殊文字があればクォート
    if re.search(r"\s", s) or re.search(r"[\\$`\"'|;&<>]", s):
        return dq(s)
    return s


In [ ]:
# ===== 入力読み込み =====
data_dir = read_first_nonempty_line(DATA_DIR_FILE)
output_dir = read_first_nonempty_line(OUTPUT_DIR_FILE)
topics_raw = read_nonempty_lines(TOPIC_FILE)

if not topics_raw:
    raise ValueError(f"{TOPIC_FILE} に topic がありません")

topics = [ensure_leading_slash(t) for t in topics_raw]

# -o で指定する json ファイル名：output_dirname の basename + .json
output_basename = Path(output_dir).name
output_json = f"{output_basename}.json"

print('--- 読み込み結果 ---')
print('QUOTE_MODE    :', QUOTE_MODE)
print('data_dir      :', data_dir)
print('topics        :', topics)
print('output_dir    :', output_dir)
print('output_json   :', output_json)


In [ ]:
# ===== コマンド生成 =====
topic_part = ' '.join(topics)

# 1行版（コピー＆ペースト用）
cmd_one_line = f"mcap_dumper {q(data_dir)} -t {topic_part} --json {OUTPUT_DIR_FLAG} {q(output_dir)} -o {q(output_json)}"

# 複数行版（見やすい、シェルで \ 改行継続できる形）
cmd_multiline = "\n".join([
    f"mcap_dumper {q(data_dir)} \\",
    f"  -t {topic_part} \\",
    f"  --json \\",
    f"  {OUTPUT_DIR_FLAG} {q(output_dir)} \\",
    f"  -o {q(output_json)}",
])

print('\n=== 生成コマンド（1行）===')
print(cmd_one_line)
print('\n=== 生成コマンド（複数行）===')
print(cmd_multiline)

# 必要なら .sh に書き出し（任意）
out_sh = Path('run_mcap_dumper.sh')
out_sh.write_text(cmd_one_line + "\n", encoding='utf-8')
print(f"\n[書き出し] {out_sh.resolve()} に 1行コマンドを保存しました")
